In [ ]:
# source tfod/bin/activate

In [ ]:
#conda info 

In [ ]:
#!pip install easyocr
#!pip install imutils

In [ ]:
#!pip install validators

In [ ]:
#!pip install matplotlib==3.6.0

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
import matplotlib

In [ ]:
import os
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

In [ ]:
print(matplotlib.__version__)

In [ ]:
import os.path
dirname_path = os.getcwd()


In [ ]:
IMAGE_PATH = os.path.join(dirname_path,'images')

In [ ]:
print(IMAGE_PATH)

In [ ]:
os.listdir()

# 1 Read, Grayscale Image

In [ ]:
cd images

In [ ]:
os.listdir()

In [ ]:
#img = cv2.imread(os.path.join(IMAGE_PATH,'car4.jpg'))
img = cv2.imread('DSC01139.JPG')

In [ ]:
print (img)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [ ]:
#2. Filtering and finding Edges for localization

In [ ]:
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction - d, sigma color, sigmaspace
plt.imshow(cv2.cvtColor(bfilter, cv2.COLOR_BGR2RGB))

In [ ]:
blurred = cv2.GaussianBlur(bfilter, (3, 3), 0)
plt.imshow(cv2.cvtColor(blurred, cv2.COLOR_BGR2RGB))

In [ ]:
u_lower = 30  # Lower Threshold
t_upper = 200  # Upper threshold
aperture_size = 5  # Aperture size
L2Gradient = True # Boolean

In [ ]:
edged = cv2.Canny(blurred, u_lower, t_upper, L2gradient = L2Gradient )  #Edge Detection
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

In [ ]:
#3. find plate contours - ie.rectangle area
# Contours is a Python list of all the contours in the image. 
# Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.

In [ ]:
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

In [ ]:
contours

In [ ]:
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break

In [ ]:
location

In [ ]:
# cv2.bitwise Arithmetic Operations on Images 

In [ ]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0,255, -1)
new_image = cv2.bitwise_and(img, img, mask=mask)

In [ ]:
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

In [ ]:
(x,y) = np.where(mask==255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
cropped_image = gray[x1:x2+1, y1:y2+1]

In [ ]:
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

In [ ]:
#4. Applying EasyOCR to Read Text

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
result